In [370]:
import pandas as pd
import numpy as np
import time

In [371]:
def reformat_date(df):
    appoint_date = []
    for i in range(len(df)):
        day = df[i]
        if pd.isna(day):
            appoint_date.append(np.nan)  
        elif day == 'REDACTED':
            appoint_date.append(np.nan)  
        elif len(day) == 8:
            if int(day[6:8])+2000 > 2021:
                appoint_date.append('19'+day[6:8]+'-'+day[0:5])
            else: 
                appoint_date.append('20'+day[6:8]+'-'+day[0:5] )
        else:
            mon = day[5:8]
            if mon == 'JAN':
                appoint_date.append(day[0:5]+'01'+day[8:])
            elif mon == 'FEB':
                appoint_date.append(day[0:5]+'02'+day[8:])
            elif mon == 'MAR':
                appoint_date.append(day[0:5]+'03'+day[8:])
            elif mon == 'APR':
                appoint_date.append(day[0:5]+'04'+day[8:])
            elif mon == 'MAY':
                appoint_date.append(day[0:5]+'05'+day[8:])
            elif mon == 'JUN':
                appoint_date.append(day[0:5]+'06'+day[8:])
            elif mon == 'JUL':
                appoint_date.append(day[0:5]+'07'+day[8:])
            elif mon == 'AUG':
                appoint_date.append(day[0:5]+'08'+day[8:])
            elif mon == 'SEP':
                appoint_date.append(day[0:5]+'09'+day[8:])
            elif mon == 'OCT':
                appoint_date.append(day[0:5]+'10'+day[8:])
            elif mon == 'NOV':
                appoint_date.append(day[0:5]+'11'+day[8:])
            elif mon == 'DEC':
                appoint_date.append(day[0:5]+'12'+day[8:])
    return appoint_date

In [372]:
df_trr = pd.read_csv("src/data/trr_trr_refresh.csv")
df_trr['trr_datetime'] = pd.to_datetime(df_trr['trr_datetime'])
df_trr['beat'] = df_trr['beat'].astype(int)
df_trr['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trr['officer_appointed_date'])).strftime('%Y-%m-%d')
df_trr['officer_birth_year'] = df_trr['officer_birth_year'].fillna(0.0).astype(int)
df_trr['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trr['officer_age'] = df_trr['officer_age'].astype(int)
df_trr['officer_on_duty'] = df_trr['officer_on_duty'].astype(bool)
df_trr['officer_injured'] = df_trr['officer_injured'].astype(bool)
df_trr['officer_in_uniform'] = df_trr['officer_in_uniform'].astype(bool)
df_trr['subject_birth_year'] = df_trr['subject_birth_year'].astype(int)
df_trr['subject_age'] = df_trr['subject_age'].astype(int)
df_trr['subject_armed'] = df_trr['subject_armed'].astype(bool)
df_trr['subject_injured'] = df_trr['subject_injured'].astype(bool)
df_trr['subject_alleged_injury'] = df_trr['subject_alleged_injury'].astype(bool)
df_trr['notify_oemc'] = df_trr['notify_oemc'].astype(bool)
df_trr['notify_district_sergeant'] = df_trr['notify_district_sergeant'].astype(bool)
df_trr['notify_op_command'] = df_trr['notify_op_command'].astype(bool)
df_trr['notify_det_division'] = df_trr['notify_det_division'].astype(bool)
df_trr['trr_created'] = pd.to_datetime(df_trr['trr_created'])

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [373]:
df_weapondischarge = pd.read_csv("src/data/trr_weapondischarge_refresh.csv")
df_weapondischarge['firearm_reloaded'] = df_weapondischarge['firearm_reloaded'].astype(bool)
df_weapondischarge['sight_used'] = df_weapondischarge['sight_used'].astype(bool)

In [375]:
df_trrstatus =pd.read_csv("src/data/trr_trrstatus_refresh.csv")
df_trrstatus['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trrstatus['officer_appointed_date'])).strftime('%Y-%m-%d')
df_trrstatus['officer_birth_year'] = df_trrstatus['officer_birth_year'].fillna(0).astype(int)
df_trrstatus['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trrstatus['status_datetime'] = pd.to_datetime(df_trrstatus['status_datetime'])

In [377]:
def reconciliation_race(df):
    race_map = dict({'AMER IND/ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'AMER INDIAN / ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'ASIAN / PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'ASIAN/PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'BLACK':'BLACK',
                'UNKNOWN':'NULL',
                 'UNKNOWN / REFUSED':'NULL',
                 'HISPANIC':'HISPANIC',
                 'WHITE':'WHITE'
                })
    for k in race_map:
        if k[0:2] == 'UN':
            df.replace(k, np.nan,inplace=True)
        else:
            df.replace(k,race_map[k] ,inplace=True)

reconciliation_race(df_trr['subject_race'])

In [378]:
def reconciliation_gender(df):
    race_map = dict({'FEMALE':'F',
                     'MALE':'M'
                })
    for k in race_map:
        df.replace(k,race_map[k] ,inplace=True)
        
reconciliation_gender(df_trr['subject_gender'])

In [379]:
def reconciliation_birth_year(df):
    birth_years = df.to_numpy()
    birth_years[birth_years < 100 ] += 1900
    birth_years[birth_years < 200 ] *= 10
    for i in range(1000,2000,100):
        birth_years[birth_years < i ] += 2000-i   
    df = pd.DataFrame(birth_years)
    
    
reconciliation_birth_year(df_trrstatus['officer_birth_year'])
reconciliation_birth_year(df_trr['subject_birth_year'])

In [380]:
# Indoor/outdoor change specific column.
df_trr.loc[(df_trr.indoor_or_outdoor == 'OUTDOOR'), 'indoor_or_outdoor'] = 'Outdoor'
df_trr.loc[(df_trr.indoor_or_outdoor == 'INDOOR'), 'indoor_or_outdoor'] = 'Indoor'

# Street. 1. change to camel case
df_trr['street'] = df_trr['street'].str.title()

# location. 1. change to camel case 2. change specific items to the correct form.
df_trr['location'] = df_trr['location'].str.title()

df_trr.loc[(df_trr.location == 'Cha Hallway / Stairwell / Elevator'), 'location'] = 'Cha Hallway/Stairwell/Elevator'
df_trr.loc[(df_trr.location == 'Cha Parking Lot / Grounds'), 'location'] = 'Cha Parking Lot/Grounds'
df_trr.loc[(df_trr.location == 'Church / Synagogue / Place Of Worship'), 'location'] = 'Church/Synagogue/Place Of Worship'
df_trr.loc[(df_trr.location == 'College / University - Grounds'), 'location'] = 'College/University Grounds'
df_trr.loc[(df_trr.location == 'Factory / Manufacturing Building'), 'location'] = 'Factory/Manufacturing Building'
df_trr.loc[(df_trr.location == 'Government Building / Property'), 'location'] = 'Government Building/Property'
df_trr.loc[(df_trr.location == 'Highway / Expressway'), 'location'] = 'Highway/Expressway'
df_trr.loc[(df_trr.location == 'Hospital Building / Grounds'), 'location'] = 'Hospital Building/Grounds'
df_trr.loc[(df_trr.location == 'Hotel / Motel'), 'location'] = 'Hotel/Motel'
df_trr.loc[(df_trr.location == 'Movie House / Theater'), 'location'] = 'Movie House/Theater'
df_trr.loc[(df_trr.location == 'Nursing / Retirement Home'), 'location'] = 'Nursing Home/Retirement Home'
df_trr.loc[(df_trr.location == 'Other (Specify)'), 'location'] = 'Other'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Parking Lot / Garage (Non Residential)'), 'location'] = 'Parking Lot/Garage(Non.Resid.)'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Police Facility / Vehicle Parking Lot'), 'location'] = 'Police Facility/Veh Parking Lot'
df_trr.loc[(df_trr.location == 'Residence - Porch / Hallway'), 'location'] = 'Residence Porch/Hallway'
df_trr.loc[(df_trr.location == 'Residence - Garage'), 'location'] = 'Residence-Garage'
df_trr.loc[(df_trr.location == 'Residence - Yard (Front / Back)'), 'location'] = 'Residential Yard (Front/Back)'
df_trr.loc[(df_trr.location == 'School - Private Building'), 'location'] = 'School, Private, Building'
df_trr.loc[(df_trr.location == 'School - Private Grounds'), 'location'] = 'School, Private, Grounds'
df_trr.loc[(df_trr.location == 'School - Public Building'), 'location'] = 'School, Public, Building'
df_trr.loc[(df_trr.location == 'School - Public Grounds'), 'location'] = 'School, Public, Grounds'
df_trr.loc[(df_trr.location == 'Sports Arena / Stadium'), 'location'] = 'Sports Arena/Stadium'
df_trr.loc[(df_trr.location == 'Tavern / Liquor Store'), 'location'] = 'Tavern/Liquor Store'
df_trr.loc[(df_trr.location == 'Vacant Lot / Land'), 'location'] = 'Vacant Lot/Land'
df_trr.loc[(df_trr.location == 'Vehicle - Other Ride Share Service (Lyft, Uber, Etc.)'), 'location'] = 'Vehicle - Other Ride Service'
df_trr.loc[(df_trr.location == 'Vehicle - Commercial'), 'location'] = 'Vehicle-Commercial'
df_trr.loc[(df_trr.location == 'Cta Parking Lot / Garage / Other Property'), 'location'] = 'Cta Garage / Other Property'
df_trr.loc[(df_trr.location == 'Lakefront / Waterfront / Riverbank'), 'location'] = 'Lakefront/Waterfront/Riverbank'
df_trr.loc[(df_trr.location == 'Medical / Dental Office'), 'location'] = 'Medical/Dental Office'
df_trr.loc[(df_trr.location == 'Airport Parking Lot'), 'location'] = 'Airport/Aircraft'
df_trr.loc[(df_trr.location == 'Airport Terminal Mezzanine - Non-Secure Area'), 'location'] = 'Airport Terminal Lower Level - Non-Secure Area'


# birth year
def age_sanity(row):
    if row < 100:
        return 1900 + row
    if 100 <= row < 1900:
        return 1900 + row % 100
    return row

df_trr.subject_birth_year = df_trr.subject_birth_year.apply(age_sanity)


In [381]:
# Read weapon file.
sub_weapon_refresh = pd.read_csv("src/data/trr_subjectweapon_refresh.csv")
# sub_weapon = pd.read_csv("../../trr_subjectweapon.csv")

# Weapon, change specific items to the correct form.
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'CHEMICAL WEAPON') | (sub_weapon_refresh.weapon_type == 'TASER / STUN GUN'), 'weapon_type'] = 'OTHER (SPECIFY)'
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'VEHICLE'), 'weapon_type'] = 'VEHICLE - ATTEMPTED TO STRIKE OFFICER WITH VEHICLE'


In [382]:
df_trr["suffix_name"] = "NULL"
df_trrstatus["suffix_name"] = "NULL"
def update_last_suffix(df):
    suffix_list = ["I", "II", "III", "IV", "V", "JR", "SR"]
    officer_last_name = []
    suffix_name = []
    for i in df['officer_last_name']:
        target_str = i.upper().split(" ")
        if len(target_str) > 1:
            k = 0
            for j in suffix_list:
                if j == ''.join(e for e in target_str[-1] if e.isalnum()):
                    # print(''.join(e for e in target_str[-1] if e.isalnum()))
                    suffix_name.append(j)
                    k +=1
            if k == 0 :
                suffix_name.append(None)
            officer_last_name.append(' '.join(target_str).title())
        else:
            officer_last_name.append(i.title())
            suffix_name.append(None)
    df['officer_last_name'] = officer_last_name
    df['suffix_name'] = suffix_name

update_last_suffix(df_trrstatus)
update_last_suffix(df_trr)

In [383]:
def update_first_name(df):
    officer_first_name = []
    for i in df['officer_first_name']:
        officer_first_name.append(i.title())
    df['officer_first_name'] = officer_first_name

update_first_name(df_trrstatus)
update_first_name(df_trr)

In [384]:
def update_race(df):
    race_map = dict({'AMER IND/ALASKAN NATIVE':'Native American/Alaskan Native',
                'ASIAN/PACIFIC ISLANDER':'Asian/Pacific',
                'BLACK':'Black',
                'UNKNOWN':'Unknown',
                 'HISPANIC':'Hispanic',
                 'WHITE':'White',
                     'WHITE HISPANIC':'Hispanic',
                     'BLACK HISPANIC':'Hispanic'
                    
                })
    for k in race_map:
        df.replace(k,race_map[k] ,inplace=True)
        
        
update_race(df_trrstatus)
update_race(df_trr)

In [385]:
df_officer = pd.read_csv("src/data/data_officer.csv")
fields = ['first_name', 'middle_initial', 'last_name',
          'suffix_name', 'birth_year', 'appointed_date', 'gender', 'race']
officer_fields = ['officer_first_name', 'officer_middle_initial', 'officer_last_name', 
          'suffix_name', 'officer_birth_year', 'officer_appointed_date', 'officer_gender', 'officer_race']

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [386]:
df_officer.fillna(value=np.nan, inplace=True)
df_trr.fillna(value=np.nan, inplace=True)

In [393]:
df_trr["officer_id"] = "NULL"

index= []
value = []
officer_id = []
for i, officer in df_trr.iterrows():
    trr = np.zeros(len(df_officer))
    for j in range(8):
        trr += (pd.isna(df_officer[fields[j]]) | pd.isna(officer[officer_fields[j]])).to_numpy()*0.9
        trr += (df_officer[fields[j]] == officer[officer_fields[j]]).to_numpy()
    temp = np.argmax(trr)
    value.append(trr[temp])
    index.append(temp)
    officer_id.append(df_officer.loc[index[i]]['id'])
    
df_trr["officer_id"]  = officer_id

In [320]:
for row in [300]:
    for j in range(8):
        print(df_officer.loc[index[row]][fields[j]] == df_trr.loc[row][officer_fields[j]])
        print("{}: {}, {}".format(fields[j], df_trr.loc[row][officer_fields[j]], df_officer.loc[index[row]][fields[j]]))

True
first_name: John, John
False
middle_initial: nan, F
True
last_name: Clark, Clark
False
suffix_name: nan, nan
False
birth_year: nan, 1970.0
True
appointed_date: 1993-05-17, 1993-05-17
True
gender: M, M
True
race: White, White


In [326]:
np.unique(value)
import collections
c = collections.Counter(value)

In [330]:
c

Counter({7.7: 6905,
         7.9: 7286,
         7.8: 1975,
         6.8: 305,
         6.699999999999999: 38,
         5.7: 17,
         5.699999999999999: 298,
         6.6: 7,
         6.7: 15,
         5.6: 8,
         6.9: 290,
         5.8: 16,
         7.0: 252,
         4.9: 41,
         5.9: 6,
         4.8: 4,
         6.0: 2})

In [388]:
df_trrstatus["officer_id"] = "NULL"

index1= []
value1 = []
officer_id1 = []
for i, officer in df_trrstatus.iterrows():
    trr = np.zeros(len(df_officer))
    for j in range(8):
        trr += (pd.isna(df_officer[fields[j]]) | pd.isna(officer[officer_fields[j]])).to_numpy()*0.9
        trr += (df_officer[fields[j]] == officer[officer_fields[j]]).to_numpy()
    temp = np.argmax(trr)
    value1.append(trr[temp])
    index1.append(temp)
    officer_id1.append(df_officer.loc[index1[i]]['id'])
    
df_trrstatus["officer_id"]  = officer_id1

In [390]:
df_trrstatus.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63455 entries, 0 to 63454
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   officer_rank              63363 non-null  object 
 1   officer_star              21933 non-null  float64
 2   status                    41430 non-null  object 
 3   status_datetime           63455 non-null  object 
 4   officer_age               22025 non-null  float64
 5   officer_first_name        63455 non-null  object 
 6   officer_middle_initial    36718 non-null  object 
 7   officer_last_name         63455 non-null  object 
 8   officer_gender            63455 non-null  object 
 9   officer_race              63455 non-null  object 
 10  officer_birth_year        41430 non-null  float64
 11  officer_appointed_date    62807 non-null  object 
 12  officer_unit_at_incident  41430 non-null  object 
 13  trr_report_id             63455 non-null  int64  
 14  suffix

In [391]:
c = collections.Counter(value)
c

Counter({7.7: 403,
         7.9: 10,
         7.8: 3,
         6.8: 19,
         6.699999999999999: 2,
         5.7: 2,
         5.699999999999999: 2,
         6.6: 1})

In [392]:
df_trrstatus.to_csv(r'1.csv',index=False)


In [394]:
df_trr.to_csv(r'2.csv',index=False)

In [367]:
df_trrstatus.loc[0]

officer_rank                                  OFFICER
officer_star                                   4502.0
status                                            NaN
status_datetime             2016-04-12 11:53:00-05:00
officer_age                                      36.0
officer_first_name                             Sylwia
officer_middle_initial                            NaN
officer_last_name                              Rejman
officer_gender                                      F
officer_race                                    White
officer_birth_year                                NaN
officer_appointed_date            2012-11-30 00:00:00
officer_unit_at_incident                          NaN
trr_report_id                                   96449
suffix_name                                      None
officer_id                                      23701
Name: 0, dtype: object